# In order to categorized product we will merge two dataframes.#
        1- df with catalogue exported from magento with sku and category
        2- df with sales by sku and client

Both dataframes are in a folder register into .gitignore

In [78]:
import pandas as pd

In [79]:
dfc = pd.read_csv('../data2/processed/catalogue_clean.csv')
dfs = pd.read_csv('../data2/processed/sales_clean.csv')

In [80]:
dfc.head(2)

,sku,attribute_set
0,MOD-110PAX4,Label Printer
1,MOD-110XiIII,Label Printer


In [81]:
dfs.head(2)

,id,vat,price_subtotal,quantity,sku
0,47053,B76645456,46152.33,30.0,MC75A6-PYCSWQRA9WR
1,32415,A48010615,43195.00,265.0,DS9808-KITSERIE


## Will Merge using the column 'sku' from both df

In [119]:
final = pd.merge(dfs,dfc, left_on='sku', right_on='sku', how = 'left')

In [120]:
final.head(2)

,id,vat,price_subtotal,quantity,sku,attribute_set
0,47053,B76645456,46152.33,30.0,MC75A6-PYCSWQRA9WR,Accessories
1,32415,A48010615,43195.00,265.0,DS9808-KITSERIE,NaN


Will drop all the registers without a valid value on 'attribute_set'

In [121]:
final['attribute_set'].isnull().sum()

16140

In [122]:
final[final['attribute_set'].isnull()]['sku'].value_counts()

Shipping and Handling    5575
Repair                    596
unknown                   304
Price-offer               184
MISCELLANEOUS              83
                         ... 
LCI-LIC                     1
PTN-T4M3YNBD                1
APN-30                      1
105909G-038                 1
SERV.CONSULT                1
Name: sku, Length: 243, dtype: int64

In [123]:
final = final[final['attribute_set'].notnull()]

In [124]:
final.shape

(9843, 6)

In [125]:
final

,id,vat,price_subtotal,quantity,sku,attribute_set
0,47053,B76645456,46152.33,30.0,MC75A6-PYCSWQRA9WR,Accessories
2,36952,B85991941,31612.15,29.0,MC75A6-PYCSWRRA9WR,Accessories
4,10894,A81939209,29424.03,1.0,MC75A6-PYCSWRRA9WR,Accessories
7,9592,A28616217,25967.06,49.0,ES405B-0AE2,Mobile Computing
9,6942,52179059V,23513.04,36.0,S4M00-200E-0100T,Label Printer
...,...,...,...,...,...,...
25727,18729,B87053047,0.00,1.0,SAMPLE5164,Accessories
25852,38144,A28250777,-20.25,1.0,LPS-MISC,Accessories
25856,38147,A28250777,-24.47,1.0,LPS-MISC,Accessories
25941,17870,A15005499,-138.23,1.0,CN70AN1KNU3W2100,Mobile Computing


Will not count lines with negative values

In [126]:
final = final[final['price_subtotal'] > 0]

In [127]:
final.shape

(9775, 6)

Will drop 'sku' column as no longer needed in order to anonymize business information

In [128]:
final = final.drop(columns=('sku'))

Will aggregate values by vat and attribute set

In [131]:
final

,id,vat,price_subtotal,quantity,attribute_set
0,47053,B76645456,46152.33,30.0,Accessories
2,36952,B85991941,31612.15,29.0,Accessories
4,10894,A81939209,29424.03,1.0,Accessories
7,9592,A28616217,25967.06,49.0,Mobile Computing
9,6942,52179059V,23513.04,36.0,Label Printer
...,...,...,...,...,...
25249,75432,B18092957,2.00,1.0,Accessories
25251,112501,B95521225,1.91,1.0,Accessories
25252,16098,43720187Q,1.91,1.0,Accessories
25266,17150,B86464922,1.59,1.0,Accessories


In [150]:
aggresults = final.groupby(['id','vat','attribute_set']).agg({'price_subtotal':['sum'], 'quantity':['sum']})

In [151]:
aggresults

,,,price_subtotal,quantity
,,,sum,sum
id,vat,attribute_set,,
125,A28415495,Label Printer,239.00,1.0
139,B12026183,Accessories,43.80,1.0
149,B64898562,Accessories,103.38,3.0
168,A59555466,Accessories,753.49,1.0
235,A58244096,Printhead,899.91,3.0
...,...,...,...,...
119242,45955229X,Other,261.31,1.0
119338,B34204974,Scanner,472.50,1.0


In [152]:
aggresults = aggresults.reset_index(level=[0,1])
aggresults = aggresults.reset_index(level=[0])

In [154]:
aggresults.columns = [col[1] for col in aggresults.columns]

In [158]:
aggresults.columns = ['category','id', 'vat', 'value', 'quantity']

In [159]:
aggresults

,category,id,vat,value,quantity
0,Label Printer,125,A28415495,239.00,1.0
1,Accessories,139,B12026183,43.80,1.0
2,Accessories,149,B64898562,103.38,3.0
3,Accessories,168,A59555466,753.49,1.0
4,Printhead,235,A58244096,899.91,3.0
...,...,...,...,...,...
9767,Other,119242,45955229X,261.31,1.0
9768,Scanner,119338,B34204974,472.50,1.0
9769,Scanner,119340,B70135371,472.50,1.0
9770,Scanner,119376,47364071X,366.41,1.0


In [160]:
df = aggresults[['id','vat','category','value','quantity']]

Will use only sales with very concrete categories dropping "Other"

In [166]:
df = df[df['category'] != 'Other']

In [167]:
df.to_csv('../data2/processed/merged_clean.csv')